In [1]:
import tensorflow as tf

import cv2
import numpy as np

import math
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

nb_classes = 10
X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,nb_classes])

W1 = tf.get_variable("w11", [784,256], initializer = tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable("w22", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer()) 
W3 = tf.get_variable("w33", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([10]))

dropout_rate = tf.placeholder("float")

_L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(_L1, dropout_rate)
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)

hypo = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = hypo, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)
is_correct = tf.equal(tf.argmax(hypo,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:
training_epochs = 20
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())
num = 20
for epoch in range(training_epochs):
    avgcost = 0
    avg=0
    total_batch = int(mnist.train.num_examples / batch_size)
        
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _= sess.run([cost,optimizer],feed_dict = {X : batch_xs, Y:batch_ys, dropout_rate : 0.7})
        avgcost += c / total_batch
          
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.5f}'.format(avgcost))
        
print("accuracy:", accuracy.eval(session = sess, feed_dict={X: mnist.test.images, Y:mnist.test.labels, dropout_rate : 1}))

Epoch: 0001 cost = 0.39674
Epoch: 0002 cost = 0.16127
Epoch: 0003 cost = 0.11792
Epoch: 0004 cost = 0.09711
Epoch: 0005 cost = 0.08420
Epoch: 0006 cost = 0.07070
Epoch: 0007 cost = 0.06313
Epoch: 0008 cost = 0.05458
Epoch: 0009 cost = 0.05045
Epoch: 0010 cost = 0.04737
Epoch: 0011 cost = 0.04400
Epoch: 0012 cost = 0.04306
Epoch: 0013 cost = 0.03616
Epoch: 0014 cost = 0.03807
Epoch: 0015 cost = 0.03481
Epoch: 0016 cost = 0.03272
Epoch: 0017 cost = 0.02963
Epoch: 0018 cost = 0.02924
Epoch: 0019 cost = 0.02728
Epoch: 0020 cost = 0.02670
accuracy: 0.9816


In [7]:
import tensorflow as tf
import cv2
import numpy as np

import math

def process(img_input):

    gray = cv2.cvtColor(img_input, cv2.COLOR_BGR2GRAY)



    cv2.imwrite('D:/deep_learning/image2.bmp', gray)
        
    (thresh, img_binary) = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    
    h,w = img_binary.shape

  
    ratio = 100/h
    new_h = 100
    new_w = w * ratio

    img_empty = np.zeros((110,110), dtype=img_binary.dtype)
    img_binary = cv2.resize(img_binary, (int(new_w), int(new_h)), interpolation=cv2.INTER_AREA)
    img_empty[:img_binary.shape[0], :img_binary.shape[1]] = img_binary

    img_binary = img_empty


    cnts = cv2.findContours(img_binary.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.imwrite('D:/deep_learning/image3.bmp', img_binary)

    # 컨투어의 무게중심 좌표를 구합니다. 
    M = cv2.moments(cnts[0][0])
    center_x = (M["m10"] / M["m00"])
    center_y = (M["m01"] / M["m00"])
    print(center_x)
    print(center_y)

    # 무게 중심이 이미지 중심으로 오도록 이동시킵니다. 
    height,width = img_binary.shape[:2]
    shiftx = width/2-center_x
    shifty = height/3-center_y

    Translation_Matrix = np.float32([[1, 0, shiftx],[0, 1, shifty]])
    img_binary = cv2.warpAffine(img_binary, Translation_Matrix, (width,height))


    img_binary = cv2.resize(img_binary, (28, 28), interpolation=cv2.INTER_AREA)
    flatten = img_binary.flatten() / 255.0
    cv2.imwrite('D:/deep_learning/image4.bmp', img_binary)

    return flatten





cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


while(True):

    ret, img_color = cap.read()

    if ret == False:
        break;

    img_input = img_color.copy()
    cv2.rectangle(img_color, (250, 150),  (width-250, height-150), (0, 0, 255), 3)
    cv2.imshow('bgr', img_color)

    img_roi = img_input[150:height-150, 250:width-250]

 
    key = cv2.waitKey(1)

    if key == 27:
        break
    elif key == 32:
        flatten = process(img_roi)
        
        images = np.zeros((1, 784))
        images[0] = flatten
        
        answer=sess.run(tf.argmax(hypo,1),feed_dict={X:images,dropout_rate : 1.0})
        print("\n answ :", answer[0])
        cv2.imwrite('D:/deep_learning/image.bmp', img_input)
        cv2.imwrite('D:/deep_learning/image1.bmp', img_roi)

        cv2.imshow('img_roi', img_roi)
        cv2.waitKey(0)
 
 
cap.release()
cv2.destroyAllWindows()

35.42230818826563
42.78143133462282

 answ : 1
34.68235294117647
23.274509803921568

 answ : 5
30.796589310829816
66.14715189873418

 answ : 3
39.2262673985039
47.56279672989184

 answ : 6
